In [67]:
import torch
import numpy as np
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from matplotlib import pyplot as plt 
import argparse

In [68]:
import tensorboardX

In [69]:
from tensorboardX import SummaryWriter
# #this allows command line arguments to be used
writer = SummaryWriter("runs/swav_test4")

In [70]:
#Image data needs to be normalized usually pixels are between 1 and 255, but we want them between 0 and 1
#we also need to turn the images into Tensors, which are similar to numpy arrays, but can work with GPUs and PyTorch functions
toTorch_normalize = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.5,),(0.5,),)])
#Load the data, both the train and test sets
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data',download=True, train=True, transform=toTorch_normalize)
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data',download=True, train=False, transform=toTorch_normalize)

In [72]:
#make a validation set
indices = list(range(len(trainset)))
np.random.shuffle(indices)
#get 20% of the train set 
split = int(np.floor(0.2 * len(trainset)))
train_sample = SubsetRandomSampler(indices[:split])
valid_sample = SubsetRandomSampler(indices[split:])

#Create Data Loader Objects
trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sample, batch_size=64)
validloader = torch.utils.data.DataLoader(trainset, sampler=valid_sample, batch_size=64)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)
xbatch_imgs, batch_labels = next(iter(trainloader))

loss:0.0014: 100%|██████████| 188/188 [00:14<00:00, 107.81it/s]

In [73]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [74]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #2D convolution layer
        self.conv1 = nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1)
        self.batchNorm2d = nn.BatchNorm2d(4) #number of features
        self.relu = nn.ReLU() #activation function -> nonlinear
        self.MaxPool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1)
        self.linear = nn.Linear(4*7*7,10) #there are 10 classes

    def forward(self, x):
        #convolution layer
        out = self.conv1(x)
        #activation function and batch norm
        activation_norm = self.relu(self.batchNorm2d(out))
        pool = self.MaxPool(activation_norm)

        out = self.conv2(pool)
        activation_norm = self.relu(self.batchNorm2d(out))
        pool = self.MaxPool(activation_norm)
        x = pool.view(pool.size(0), -1)
        probabilities = self.linear(x)
        return probabilities


In [75]:
#define the model
model = Net()
#define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#define the loss function
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)
print(model)


Net(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchNorm2d): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (MaxPool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear): Linear(in_features=196, out_features=10, bias=True)
)


In [76]:
num_epochs = 1
mean_loss = []
accuracy_list = []
mean_loss_val = []
loop = tqdm(total=num_epochs*len(trainloader), position=0)

writer = SummaryWriter("/Users/raelynnwonnacott/Desktop/AF/runs/proj_test5")

for epoch_number in range(num_epochs):
    print(epoch_number)
    model = model.train()
    epoch_loss = 0
    #initialize lists to store embeddings and labels
    tensor_list = []
    label_list = []
    #Loop through the data for training
    for it, input in enumerate(trainloader):
        #clear the gradients of the model parameters
        optimizer.zero_grad()
        #putting data on "device"
        x, y = x.to(device), y.to(device)
        model.zero_grad()
        # pass data through the model
        output = model(x)
        # add features to list
        tensor_list.append(output)
        label_list.append(y)
        loss = criterion(output, y)
        loss.backward() #Compute gradients
        epoch_loss += loss.item()
        
        optimizer.step() #Optimize and update the weights
        loop.set_description('loss:{:.4f}'.format(loss.item()))
        loop.update()
    torch.save({
        'epoch': epoch_number,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,},"/Users/raelynnwonnacott/Desktop/AF/models")
    
    #add embeddings
    if epoch_number % 10 == 0:
        writer.add_embedding(torch.cat(tensor_list),metadata = torch.cat(label_list),global_step = it)
    
    mean_loss.append(epoch_loss/len(trainloader))
    
#     accuracy = 0
#     for batch, (x, y_truth) in enumerate(testloader):
#         x, y_truth = x.to(device), y_truth.to(device)
#         output = model(x)
#         y_hat = torch.argmax(output, axis=1)

#         accuracy += torch.sum(y_hat == y_truth).item()
#         #print(accuracy)
#     accuracy_list.append(accuracy/len(testloader))

loss:0.3099:   9%|▊         | 16/188 [00:00<00:02, 65.49it/s]

0


loss:0.0028:  99%|█████████▉| 187/188 [00:01<00:00, 101.49it/s]

loss:0.0028: 100%|██████████| 188/188 [00:18<00:00, 101.49it/s]